In [69]:
from dotenv import load_dotenv
from llama_index.llms.openrouter import OpenRouter
from llama_index.core.llms import ChatMessage
from os import getenv

load_dotenv()

True

In [70]:
llm = OpenRouter(
    max_tokens=256,
    context_window=4096,
    model="cohere/command-r"
)

In [60]:
message = ChatMessage(role="user", content="Tell me a joke")
resp = llm.chat([message])
print(resp)

assistant: What do you call a factory that makes distinctly average products?

*A satisfactory.*


In [71]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader



In [72]:
documents = SimpleDirectoryReader("poems_data").load_data()

Trying out Cohere embeddings

In [73]:
from llama_index.embeddings.cohere import CohereEmbedding

embed_model = CohereEmbedding(
    api_key=getenv("COHERE_API_KEY"),
    model_name="embed-multilingual-light-v3.0",
    input_type="search_query",
)

In [74]:
from llama_index.llms.cohere import Cohere

llm_original = Cohere(model="command-r", api_key=getenv("COHERE_API_KEY"))

In [40]:
llm.chat([ChatMessage(role="user", content="Rancontrez-moi une blague")])

ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="Je suis heureux de pouvoir vous aider! Mais, s'il vous plaît, écrivez une blague et je suis heureux de le contrer.\n\n(In English, please) \n\nLooking forward to trading jokes with you!", additional_kwargs={}), raw={'text': "Je suis heureux de pouvoir vous aider! Mais, s'il vous plaît, écrivez une blague et je suis heureux de le contrer.\n\n(In English, please) \n\nLooking forward to trading jokes with you!", 'generation_id': '01246f4b-e584-4b90-9f88-a3b1bef31356', 'citations': None, 'documents': None, 'is_search_required': None, 'search_queries': None, 'search_results': None, 'finish_reason': 'COMPLETE', 'tool_calls': None, 'chat_history': [Message_User(message='Rancontrez-moi une blague', tool_calls=None, role='USER'), Message_Chatbot(message="Je suis heureux de pouvoir vous aider! Mais, s'il vous plaît, écrivez une blague et je suis heureux de le contrer.\n\n(In English, please) \n\nLooking forward 

In [75]:
from llama_index.core import Settings

Settings.llm=llm
Settings.embed_model=embed_model

In [76]:
index = VectorStoreIndex.from_documents(
    documents=documents,
)

In [77]:
from llama_index.postprocessor.cohere_rerank import CohereRerank

cohere_rerank = CohereRerank()

In [78]:
query_engine = index.as_query_engine(
    postprocessors=[cohere_rerank],
)

In [90]:
response = query_engine.query("A qui semble le poete?")

In [91]:
response

Response(response='Le poète semble être comparable au prince des nuées.', source_nodes=[NodeWithScore(node=TextNode(id_='e12f61eb-7210-4d9a-9b1d-6041fcf74e70', embedding=None, metadata={'file_path': '/home/tphung/projects/lawrag/data/albatros.txt', 'file_name': 'albatros.txt', 'file_type': 'text/plain', 'file_size': 762, 'creation_date': '2024-08-05', 'last_modified_date': '2024-08-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='cd24558d-1689-4b77-9e6f-4a679a9e39ae', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/home/tphung/projects/lawrag/data/albatros.txt', 'file_name': 'albatros.txt', 'file_type': 'text/plain', 'file_size': 762, 'creation_date': '2024-08-05', 'last_modified_date': 

example of Vectore storage persisting

In [101]:
index.storage_context.persist(persist_dir="./poems_storage") # save the index to ./storage directory

In [102]:
from llama_index.core import load_index_from_storage, StorageContext
from llama_index.core.storage.docstore import SimpleDocumentStore
from llama_index.core.storage.index_store import SimpleIndexStore
from llama_index.core.vector_stores import SimpleVectorStore

storage_context = StorageContext.from_defaults(
    persist_dir="poems_storage"
    # theese arguments do not work but they were in guide
    # docstore=SimpleDocumentStore.from_persist_dir(persist_dir="storage"),
    # index_store=SimpleIndexStore.from_persist_dir(persist_dir="storage"),

    # vector_store=SimpleVectorStore.from_persist_dir(persist_dir="storage"),
)
persistent_index = load_index_from_storage(storage_context=storage_context)

In [103]:
persistent_engine = persistent_index.as_query_engine(
    postprocessors=[cohere_rerank],
)

In [104]:
response_new = persistent_engine.query("A qui semble le poete?")
response_new

Response(response='Le poète semble être comparable au prince des nuées.', source_nodes=[NodeWithScore(node=TextNode(id_='e12f61eb-7210-4d9a-9b1d-6041fcf74e70', embedding=None, metadata={'file_path': '/home/tphung/projects/lawrag/data/albatros.txt', 'file_name': 'albatros.txt', 'file_type': 'text/plain', 'file_size': 762, 'creation_date': '2024-08-05', 'last_modified_date': '2024-08-05'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='cd24558d-1689-4b77-9e6f-4a679a9e39ae', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/home/tphung/projects/lawrag/data/albatros.txt', 'file_name': 'albatros.txt', 'file_type': 'text/plain', 'file_size': 762, 'creation_date': '2024-08-05', 'last_modified_date': 